**INSTALL DEPENDEINCES**

In [ ]:
!pip install datasets pandas torch datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

This script trains a T5-based model to generate test cases and expected outcomes from feature descriptions. It takes a dataset containing feature descriptions, test cases, and expected outcomes, processes it, and fine-tunes the T5 model for text generation.

In [ ]:
import pandas as pd
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset

# ✅ Load Data
df = pd.read_excel("/content/Training_data_latest.xlsx", engine="openpyxl")

# ✅ Ensure Data Is Clean
df = df.dropna(subset=["Feature Description", "Test Cases", "Expected Outcome"])

# ✅ Add Task Prefix
df["Feature Description"] = "generate test case: " + df["Feature Description"].astype(str)

# ✅ Combine Test Cases & Expected Outcomes
df["Test Cases"] = df["Test Cases"].astype(str)
df["Expected Outcome"] = df["Expected Outcome"].astype(str)
df["train_labels"] = df["Test Cases"] + " | " + df["Expected Outcome"]

# ✅ Convert to Lists for Tokenization
train_texts = df["Feature Description"].tolist()
train_labels = df["train_labels"].tolist()

# ✅ Load T5 Model & Tokenizer
model_name = "t5-base"  # Change to "t5-base" for better results

tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# ✅ Tokenize Data
def tokenize_function(examples):
    model_inputs = tokenizer(examples["Feature Description"], truncation=True, padding="max_length", max_length=512)
    labels = tokenizer(examples["train_labels"], truncation=True, padding="max_length", max_length=512)

    labels["input_ids"] = [[(token if token != tokenizer.pad_token_id else -100) for token in seq] for seq in labels["input_ids"]]
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# ✅ Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df)
dataset = dataset.map(tokenize_function, batched=True)
dataset = dataset.remove_columns(["Feature Description", "Test Cases", "Expected Outcome", "train_labels"])
dataset.set_format("torch")

# ✅ Define Training Arguments
training_args = TrainingArguments(
    output_dir="./model_output",
    evaluation_strategy="no",  # No eval dataset yet
    save_strategy="epoch",
    per_device_train_batch_size=8,
    num_train_epochs=5,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    fp16=True  # ✅ Enables mixed precision for faster training
)

# ✅ Train Model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset
)

trainer.train()

# ✅ Save Model
model.save_pretrained("/content/drive/MyDrive/test_case_model")
tokenizer.save_pretrained("/content/drive/MyDrive/test_case_model")

print("✅ Model Training Complete & Saved!")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


...

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: narenminukuri (narenminukuri-home) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
10,3.989400
20,2.556300
30,1.563800
40,0.869000
50,0.472300
60,0.297100
70,0.213600
80,0.162600
90,0.156600
100,0.135200


✅ Model Training Complete & Saved!


Script to predict context aware test case usinf pretrained

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

# Load the trained model & tokenizer
model_path = "/content/drive/MyDrive/test_case_model"  # Change if saved elsewhere
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [ ]:
import random

def generate_unique_test_cases(feature_description, num_cases=4, max_length=512, max_attempts=10):
    input_text = "generate test case and expected outcome: " + feature_description  # ✅ Explicit prompt

    # Tokenize input
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {key: val.to(device) for key, val in inputs.items()}  # Move to GPU if available

    unique_test_cases = set()  # ✅ Store unique test cases
    attempts = 0

    while len(unique_test_cases) < num_cases and attempts < max_attempts:
        with torch.no_grad():
            output = model.generate(
                **inputs,
                max_length=max_length,
                num_return_sequences=num_cases,
                temperature=0.7,
                num_beams=10,  # Higher beam count for diverse outputs
                early_stopping=True
            )

        # Decode and filter duplicates
        for out in output:
            decoded_tc = tokenizer.decode(out, skip_special_tokens=True)
            if decoded_tc not in unique_test_cases:
                unique_test_cases.add(decoded_tc)

        attempts += 1  # Prevent infinite loops

    return list(unique_test_cases)[:num_cases]  # ✅ Return exactly 'num_cases' unique test cases



In [ ]:
def process_test_case_output(test_case_output):
    if "|" in test_case_output:
        test_case, expected_outcome = test_case_output.split("|", 1)
        return test_case.strip(), expected_outcome.strip()
    else:
        return test_case_output.strip(), "Expected outcome not generated, manually verify."

# ✅ Feature Description
feature_description = """The system automatically monitors financial regulatory updates from global and local authorities. It integrates with regulatory data sources to fetch and analyze changes in compliance requirements, ensuring financial institutions remain compliant. The system updates internal policies, modifies transaction validation rules, and alerts compliance officers about critical regulatory changes. Additionally, it enforces regulatory reporting and maintains an audit trail for all changes."""

# ✅ Generate test cases
test_cases = generate_unique_test_cases(feature_description, num_cases=5)

# ✅ Display output with expected outcome handling
print("Generated Test Cases & Expected Outcomes:\n")
for i, tc in enumerate(test_cases, 1):
    test_case, expected_outcome = process_test_case_output(tc)
    print(f"Test Case {i}: {test_case}\nExpected Outcome: {expected_outcome}\n")

Generated Test Cases & Expected Outcomes:

Test Case 1: Test whether unauthorized access is possible for financial institutions.
Expected Outcome: Access is restricted, and unauthorized attempts are logged.

Test Case 2: Ensure compliance officers are notified about regulatory updates.
Expected Outcome: System detects and prevents unauthorized modifications.

Test Case 3: Simulate an attempt to bypass regulatory requirements.
Expected Outcome: System detects and prevents unauthorized modifications.

Test Case 4: Simulate an attempt to bypass regulatory updates.
Expected Outcome: System detects and prevents unauthorized modifications.

Test Case 5: Ensure compliance officers are notified of regulatory updates.
Expected Outcome: System detects and prevents unauthorized modifications.

